In [7]:
!pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.6/95.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00


In [8]:
!pip install frontend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00


In [9]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 84.7 MB/s eta 0:00:00


In [ ]:
import fitz  # PyMuPDF for reading PDFs

# Function to extract text from PDF
def extract_text_from_pdf(file_path):
    document = fitz.open(file_path)
    text = ""
    for page in document:
        text += page.get_text()
    document.close()
    return text

# Paths to your PDF files
pdf_paths = [
    '/content/Infosys_2021.pdf',
    '/content/Infosys_2022.pdf',
    '/content/Infosys_2023.pdf',
    '/content/sa-fy23-annual-finstatement.pdf'
]

# Extract and print raw text from each PDF
for i, pdf_path in enumerate(pdf_paths):
    print(f"Raw text from PDF {i+1} ({pdf_path}):\n")
    raw_text = extract_text_from_pdf(pdf_path)
    print(raw_text[:2000])  # Display the first 2000 characters of the text
    print("\n" + "="*80 + "\n")  # Separator between PDFs


In [ ]:
import fitz  # PyMuPDF
import re
import pandas as pd

def extract_text_from_pdf(file_path):
    """Extract text from each page of the given PDF."""
    document = fitz.open(file_path)
    text = ""
    for page in document:
        text += page.get_text()
    document.close()
    return text

def parse_financial_data(text):
    """Parse financial data from extracted text to create a structured DataFrame."""
    lines = text.splitlines()
    data = []
    description = ""
    note_no = ""

    for line in lines:
        line = line.strip()
        # Regex to find numeric patterns that may include commas
        amounts = re.findall(r'\d{1,3}(?:,\d{3})*', line)

        if amounts:
            if len(amounts) >= 2:  # Ensure there are at least two amounts to consider them as 2020 and 2021 data
                # Only consider the first two amounts
                amount_2020, amount_2021 = amounts[:2]
                if description:  # Ensure there is a description to pair with the amounts
                    data.append([note_no, description.strip(), amount_2020, amount_2021])
                description = ""  # Reset description for the next entry
        else:
            note_match = re.match(r'(\d+\.\d+)', line)
            if note_match:
                if description and note_no:
                    data.append([note_no, description.strip(), "", ""])  # Append with empty amounts if no numeric data followed
                note_no = note_match.group(1)
                description = line.replace(note_no, '').strip()
            else:
                description += " " + line  # Continue building the description

    # Create DataFrame with specific column names for the years
    df = pd.DataFrame(data, columns=["Note No.", "Description", "Amount 2020", "Amount 2021"])
    return df

# Specify the PDF file
pdf_path = '/content/Consolidated and Standalone.pdf'  # Adjust the path to your specific file

# Extract text and parse financial data
text = extract_text_from_pdf(pdf_path)
df = parse_financial_data(text)

# Display the parsed data
print(df.head(20))


   Note No.                                        Description Amount 2020  \
0                                                   Year ended         202   
1                                      Revenue from operations      45,411   
2                                                 Other income         721   
3                                                 TOTAL INCOME      46,132   
4                           Expenses Employee benefit expenses      25,649   
5                                 Fees to external consultants       3,900   
6                      Cost of equipment and software licences         258   
7                        Depreciation and amortisation expense       1,075   
8                                               Other expenses       2,941   
9                                               TOTAL EXPENSES      33,823   
10           PROFIT BEFORE FINANCE COSTS, EXCEPTIONAL ITEM ...      12,309   
11                                               Finance costs  

In [ ]:
# Save the DataFrame to a CSV file
csv_path = '/content/parsed_financial_data.csv'
df.to_csv(csv_path, index=False)
print(f"Data saved to {csv_path}")


Data saved to /content/parsed_financial_data.csv


In [40]:
import pandas as pd

infosys_files = [
    '/content/infosys 2021.csv',
    '/content/infosys 2022.csv',
    '/content/infosys 2023.csv',
    '/content/infosys fy 2023.csv'
]

tcs_files = [
    '/content/TCS 2020.csv',
    '/content/TCS 2022.csv',
    '/content/TCS 2020-2021.csv'
]

# Function to load and display column headers from each file
def display_headers(file_list, company_name):
    print(f"Headers for {company_name} files:")
    for file_path in file_list:
        try:
            # Read only the first row to get the headers
            df = pd.read_csv(file_path, nrows=0)  # nrows=0 loads no data rows, just headers
            print(f"\nHeaders from {file_path.split('/')[-1]}:")
            print(df.columns.tolist())  # Display the list of column names
        except Exception as e:
            print(f"Failed to read headers from {file_path}: {e}")

# Display headers for Infosys and TCS files
display_headers(infosys_files, "Infosys")
display_headers(tcs_files, "TCS")


Headers for Infosys files:

Headers from infosys 2021.csv:
['Note No.', 'Description', 'Amount 2020', 'Amount 2021']

Headers from infosys 2022.csv:
['Note No.', 'Description', 'Amount 2021', 'Amount 2022']

Headers from infosys 2023.csv:
['Note No.', 'Description', 'Amount 2022', 'Amount 2023']

Headers from infosys fy 2023.csv:
['Note No.', 'Description', 'Amount 2022', 'Amount 2023']
Headers for TCS files:

Headers from TCS 2020.csv:
['Note No.', 'Description', 'Amount 2019', 'Amount 2020']

Headers from TCS 2022.csv:
['Note No.', 'Description', 'Amount 2021', 'Amount 2022']

Headers from TCS 2020-2021.csv:
['Note No.', 'Description', 'Amount 2020', 'Amount 2021']


In [3]:
# @title Install libraries for setting up for tarning using LLM
# Install transformers and any other necessary libraries
!pip install transformers torch --upgrade


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [41]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [42]:
from torch.optim import AdamW
import pandas as pd
from torch.utils.data import TensorDataset, DataLoader, RandomSampler
from sklearn.model_selection import train_test_split
import torch
import fitz
import pandas as pd
import re
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [43]:
# @title Load a Pre-trained Model and Tokenizer
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to(device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [44]:
import pandas as pd

# Dictionary to hold DataFrames keyed by year/company
dataframes = {
    'Infosys 2021': pd.read_csv('/content/infosys 2021.csv'),
    'Infosys 2022': pd.read_csv('/content/infosys 2022.csv'),
    'Infosys 2023': pd.read_csv('/content/infosys 2023.csv'),
    'Infosys FY 2023': pd.read_csv('/content/infosys fy 2023.csv'),
    'TCS 2020': pd.read_csv('/content/TCS 2020.csv'),
    'TCS 2022': pd.read_csv('/content/TCS 2022.csv'),
    'TCS 2020-2021': pd.read_csv('/content/TCS 2020-2021.csv')
}

# Example usage: print the head of the Infosys 2021 data
print(dataframes['Infosys 2022'].head())


   Note No.                            Description Amount 2021 Amount 2022
0       NaN  INFOSYS LIMITED (In ₹ crore) Note No.          31         202
1       NaN          Property, plant and equipment           2           1
2       NaN                    Right-of-use assets           2           3
3       NaN               Capital work-in-progress         411         906
4       NaN                               Goodwill           2           2


In [45]:
import pandas as pd

# Dictionary setup
dataframes = {
    'Infosys 2021': pd.read_csv('/content/infosys 2021.csv'),
    'Infosys 2022': pd.read_csv('/content/infosys 2022.csv'),
    'Infosys 2023': pd.read_csv('/content/infosys 2023.csv'),
    'Infosys FY 2023': pd.read_csv('/content/infosys fy 2023.csv'),
    'TCS 2020': pd.read_csv('/content/TCS 2020.csv'),
    'TCS 2022': pd.read_csv('/content/TCS 2022.csv'),
    'TCS 2020-2021': pd.read_csv('/content/TCS 2020-2021.csv')
}

# Initialize a dictionary to hold the labels for each year
combined_labels = {}

# Preprocess data and generate labels for each year present in the DataFrames
for key, df in dataframes.items():
    # Detect amount columns present in the current DataFrame
    amount_columns = [col for col in df.columns if 'Amount' in col]
    # Preprocess each amount column found
    for column in amount_columns:
        # Remove commas and convert to numeric
        df[column] = pd.to_numeric(df[column].replace(',', '', regex=True), errors='coerce')
        # Create labels based on a threshold (e.g., 1000)
        labels = df[column].apply(lambda x: 1 if x > 1000 else 0).tolist()
        # Store the labels with a key indicating the DataFrame and year
        combined_labels[f'labels_{column}_{key}'] = labels

    # Update the dictionary with the modified DataFrame
    dataframes[key] = df

#  printing the modified DataFrame and labels
print(dataframes['Infosys 2021'].head())
print(combined_labels.get('labels_Amount 2021_Infosys 2021'))
print(dataframes['Infosys 2022'].head())
print(combined_labels.get('labels_Amount 2022_Infosys 2022'))
print(dataframes['Infosys 2023'].head())
print(combined_labels.get('labels_Amount 2023_Infosys 2023'))
print(dataframes['Infosys FY 2023'].head())
print(combined_labels.get('labels_Amount 2023_Infosys FY  2023'))
print(dataframes['TCS 2020'].head())
print(combined_labels.get('labels_Amount 2020_TCS 2020'))
print(dataframes['TCS 2022'].head())
print(combined_labels.get('labels_Amount 2022_TCS 2022'))
print(dataframes['TCS 2020-2021'].head())
print(combined_labels.get('labels_Amount 2020_TCS 2020-2021'))

   Note No.                            Description  Amount 2020  Amount 2021
0       NaN  INFOSYS LIMITED (In ₹ crore) Note No.           31          202
1       NaN          Property, plant and equipment            2            1
2       NaN                    Right-of-use assets            2            2
3       NaN               Capital work-in-progress          906          945
4       NaN                               Goodwill          167           29
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
   Note No.                            Description  Amount 2021  Amount 2022
0       NaN  INFOSYS LIMITED (In ₹ crore) Note No.           31          202
1       NaN          Property, plant and equipment            2            1
2       NaN          

In [46]:
import pandas as pd
dataframes = {
    'Infosys 2021': pd.read_csv('/content/infosys 2021.csv'),
    'Infosys 2022': pd.read_csv('/content/infosys 2022.csv'),
    'Infosys 2023': pd.read_csv('/content/infosys 2023.csv'),
    'Infosys FY 2023': pd.read_csv('/content/infosys fy 2023.csv'),
    'TCS 2020': pd.read_csv('/content/TCS 2020.csv'),
    'TCS 2022': pd.read_csv('/content/TCS 2022.csv'),
    'TCS 2020-2021': pd.read_csv('/content/TCS 2020-2021.csv')
}

# Iterate over each DataFrame to print details for any columns that include 'Amount'
for key, df in dataframes.items():
    # Identify amount columns present in the DataFrame
    amount_columns = [col for col in df.columns if 'Amount' in col]
    # Continue only if there are amount columns in the DataFrame
    if amount_columns:
        # Check the data types of the Amount columns
        print(f"Data Types for {key}:")
        print(df[amount_columns].dtypes)

        # Display the first few rows of the Amount columns
        print(f"\nFirst Few Rows for {key}:")
        print(df[amount_columns].head())

        # Display descriptive statistics for the Amount columns
        print(f"\nDescriptive Statistics for {key}:")
        print(df[amount_columns].describe())

        # Add a separator for readability between different data outputs
        print("\n" + "-"*50 + "\n")


Data Types for Infosys 2021:
Amount 2020    object
Amount 2021    object
dtype: object

First Few Rows for Infosys 2021:
  Amount 2020 Amount 2021
0          31         202
1           2           1
2           2           2
3         906         945
4         167          29

Descriptive Statistics for Infosys 2021:
       Amount 2020 Amount 2021
count           95          95
unique          42          54
top              2         202
freq            39           7

--------------------------------------------------

Data Types for Infosys 2022:
Amount 2021    object
Amount 2022    object
dtype: object

First Few Rows for Infosys 2022:
  Amount 2021 Amount 2022
0          31         202
1           2           1
2           2           3
3         411         906
4           2           2

Descriptive Statistics for Infosys 2022:
       Amount 2021 Amount 2022
count           96          96
unique          40          55
top              2         202
freq            42           8

In [47]:
import pandas as pd

# Dictionary setup
dataframes = {
    'Infosys 2021': pd.read_csv('/content/infosys 2021.csv'),
    'Infosys 2022': pd.read_csv('/content/infosys 2022.csv'),
    'Infosys 2023': pd.read_csv('/content/infosys 2023.csv'),
    'Infosys FY 2023': pd.read_csv('/content/infosys fy 2023.csv'),
    'TCS 2020': pd.read_csv('/content/TCS 2020.csv'),
    'TCS 2022': pd.read_csv('/content/TCS 2022.csv'),
    'TCS 2020-2021': pd.read_csv('/content/TCS 2020-2021.csv')
}

# Initialize lists to collect all descriptions and amounts
descriptions = []
amounts_dict = {}

# Iterate through each DataFrame
for key, df in dataframes.items():
    # Handle NaN values: Replace NaN with 0 for relevant Amount columns
    amount_columns = [col for col in df.columns if 'Amount' in col]
    df[amount_columns] = df[amount_columns].fillna(0)

    # Append descriptions
    descriptions.extend(df['Description'].tolist())

    # Collect amounts from each column
    for column in amount_columns:
        if column not in amounts_dict:
            amounts_dict[column] = []
        amounts_dict[column].extend(df[column].tolist())

# Ensure all lists are of the same length
max_length = len(descriptions)
for amount_col, amounts in amounts_dict.items():
    if len(amounts) != max_length:
        # Pad the amounts list with zeros if they are not of the same length
        amounts.extend([0] * (max_length - len(amounts)))

# Create the DataFrame
response_df = pd.DataFrame({
    'Description': descriptions
})
for amount_col, amounts in amounts_dict.items():
    response_df[amount_col] = amounts

# Print the first 20 entries to verify
print(response_df.head(20))


                                    Description Amount 2020 Amount 2021  \
0         INFOSYS LIMITED (In ₹ crore) Note No.          31         202   
1                 Property, plant and equipment           2           1   
2                           Right-of-use assets           2           2   
3                      Capital work-in-progress         906         945   
4                                      Goodwill         167          29   
5                       Other intangible assets          67          48   
6                  Financial assets Investments           2           3   
7                                         Loans           2           4   
8                        Other financial assets           2           5   
9                     Deferred tax assets (net)         955       1,429   
10                      Income tax assets (net)       5,287       4,773   
11                     Other non-current assets           2           8   
12  Current assets Financ

In [48]:
# Ensure all descriptions are strings and handle None values
descriptions = [str(desc) if desc is not None else "" for desc in descriptions]

# Tokenize the descriptions again
try:
    encodings = tokenizer(descriptions, truncation=True, padding=True, max_length=128, return_tensors="pt")
    input_ids = encodings['input_ids'].to(device)
    attention_mask = encodings['attention_mask'].to(device)
except Exception as e:
    print("An error occurred during tokenization:", e)


In [49]:
# @title Tokenize Text Descriptions
from transformers import AutoTokenizer
import torch

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# Check if GPU is available and set it as the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tokenize the descriptions
encodings = tokenizer(descriptions, truncation=True, padding=True, max_length=128, return_tensors="pt")

# Convert encodings to tensors and move them to the device
input_ids = encodings['input_ids'].to(device)
attention_mask = encodings['attention_mask'].to(device)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [50]:
# @title Create Dataset and DataLoader
from torch.utils.data import DataLoader, TensorDataset
import torch

# Ensure all amounts are converted to floats, handling non-numeric values
amounts_2019_tensor = torch.tensor(response_df['Amount 2019'].apply(pd.to_numeric, errors='coerce').fillna(0).values, dtype=torch.float).to(device)
amounts_2020_tensor = torch.tensor(response_df['Amount 2020'].apply(pd.to_numeric, errors='coerce').fillna(0).values, dtype=torch.float).to(device)
amounts_2021_tensor = torch.tensor(response_df['Amount 2021'].apply(pd.to_numeric, errors='coerce').fillna(0).values, dtype=torch.float).to(device)
amounts_2022_tensor = torch.tensor(response_df['Amount 2022'].apply(pd.to_numeric, errors='coerce').fillna(0).values, dtype=torch.float).to(device)
amounts_2023_tensor = torch.tensor(response_df['Amount 2023'].apply(pd.to_numeric, errors='coerce').fillna(0).values, dtype=torch.float).to(device)

In [51]:
# Create the TensorDataset including all years
dataset = TensorDataset(input_ids, attention_mask, amounts_2019_tensor, amounts_2020_tensor, amounts_2021_tensor, amounts_2022_tensor, amounts_2023_tensor)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [52]:
import torch.nn as nn

def custom_loss_function(outputs, *targets):
    mse_loss = nn.MSELoss()
    losses = []
    # Check if outputs dimension matches the number of targets
    if outputs.shape[1] != len(targets):
        raise ValueError(f"Output dimensions {outputs.shape[1]} do not match the number of targets {len(targets)}")

    for i, target in enumerate(targets):
        loss = mse_loss(outputs[:, i], target)
        losses.append(loss)

    total_loss = sum(losses) / len(losses)
    return total_loss


In [54]:
from transformers import BertConfig, BertForSequenceClassification

# Load the configuration from the pre-trained BERT model
config = BertConfig.from_pretrained('bert-base-uncased')

# Adjust the number of labels to match the number of output years (2019-2023)
config.num_labels = 5  # This should match the number of years you are predicting

# Initialize the model with the custom configuration
model = BertForSequenceClassification(config)
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [56]:
# @title Define the Training Loop
from torch.optim import AdamW

# Initialize the optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

for epoch in range(4):  # Number of epochs
    total_loss = 0
    for batch in dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_attention_mask, *b_amounts = batch  # Unpack all amounts

        model.zero_grad()

        # Forward pass
        outputs = model(b_input_ids, attention_mask=b_attention_mask).logits
        loss = custom_loss_function(outputs, *b_amounts)

        total_loss += loss.item()
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch + 1} | Average Loss: {total_loss / len(dataloader)}')



Epoch 1 | Average Loss: 16086.642919218386
Epoch 2 | Average Loss: 15751.228641658634
Epoch 3 | Average Loss: 15538.279510696213
Epoch 4 | Average Loss: 15482.218416387384


Evaluate the model

In [57]:
from sklearn.model_selection import train_test_split

# Assuming response_df is your full dataset containing all descriptions and amounts
train_df, test_df = train_test_split(response_df, test_size=0.2, random_state=42)  # Adjust test_size as needed

# Separate descriptions for tokenizing
train_texts = train_df['Description'].tolist()
test_texts = test_df['Description'].tolist()


In [58]:
# Ensure all descriptions are strings and handle None values
test_texts = [str(text) if text is not None else "" for text in test_texts]

# Retry tokenizing the test data
try:
    test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt")
    test_input_ids = test_encodings['input_ids'].to(device)
    test_attention_mask = test_encodings['attention_mask'].to(device)
except Exception as e:
    print("Failed during tokenization:", e)


In [59]:
# Convert columns to numeric, coercing errors and filling NaNs
test_df['Amount 2019'] = pd.to_numeric(test_df['Amount 2019'], errors='coerce').fillna(0)
test_df['Amount 2020'] = pd.to_numeric(test_df['Amount 2020'], errors='coerce').fillna(0)
test_df['Amount 2021'] = pd.to_numeric(test_df['Amount 2021'], errors='coerce').fillna(0)
test_df['Amount 2022'] = pd.to_numeric(test_df['Amount 2022'], errors='coerce').fillna(0)
test_df['Amount 2023'] = pd.to_numeric(test_df['Amount 2023'], errors='coerce').fillna(0)

# Convert test amounts to tensors
try:
    test_amounts_2019 = torch.tensor(test_df['Amount 2019'].values, dtype=torch.float).to(device)
    test_amounts_2020 = torch.tensor(test_df['Amount 2020'].values, dtype=torch.float).to(device)
    test_amounts_2021 = torch.tensor(test_df['Amount 2021'].values, dtype=torch.float).to(device)
    test_amounts_2022 = torch.tensor(test_df['Amount 2022'].values, dtype=torch.float).to(device)
    test_amounts_2023 = torch.tensor(test_df['Amount 2023'].values, dtype=torch.float).to(device)
except TypeError as e:
    print("Failed to convert amounts to tensors:", e)


In [60]:
print(test_df[['Amount 2019', 'Amount 2020', 'Amount 2021', 'Amount 2022', 'Amount 2023']].dtypes)


Amount 2019    float64
Amount 2020    float64
Amount 2021    float64
Amount 2022    float64
Amount 2023    float64
dtype: object


In [61]:
# Create the TensorDataset for the test set
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_amounts_2019, test_amounts_2020, test_amounts_2021, test_amounts_2022, test_amounts_2023)

# Create DataLoader for the test set
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [62]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from transformers import AutoTokenizer

# Assuming the model and custom_loss_function are already defined
model.eval()  # Set the model to evaluation mode

# Initialize test DataLoader
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)  # Make sure test_dataset is defined as shown previously

# Disable gradient calculation for evaluation
total_eval_loss = 0
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the appropriate device
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_attention_mask, b_amounts_2019, b_amounts_2020, b_amounts_2021, b_amounts_2022, b_amounts_2023 = batch

        # Forward pass: compute predicted outputs by passing inputs to the model
        outputs = model(b_input_ids, attention_mask=b_attention_mask).logits
        loss = custom_loss_function(outputs, b_amounts_2019, b_amounts_2020, b_amounts_2021, b_amounts_2022, b_amounts_2023)

        # Update test loss
        total_eval_loss += loss.item()

# Calculate the average loss over all of the batches
average_eval_loss = total_eval_loss / len(test_loader)
print(f"Average Evaluation Loss: {average_eval_loss:.4f}")


Average Evaluation Loss: 16083.3772


In [67]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [68]:
# @title Saving the Model
model_path = '/content/drive/My Drive/My Models/bert_model_infosys+TCS'
tokenizer_path = '/content/drive/My Drive/My Models/bert_model_infosys+TCS'

model.save_pretrained(model_path)
tokenizer.save_pretrained(tokenizer_path)


('/content/drive/My Drive/My Models/bert_model_infosys+TCS/tokenizer_config.json',
 '/content/drive/My Drive/My Models/bert_model_infosys+TCS/special_tokens_map.json',
 '/content/drive/My Drive/My Models/bert_model_infosys+TCS/vocab.txt',
 '/content/drive/My Drive/My Models/bert_model_infosys+TCS/added_tokens.json',
 '/content/drive/My Drive/My Models/bert_model_infosys+TCS/tokenizer.json')

In [69]:
import os
import torch

# Ensure the directory exists
model_dir = './model'
os.makedirs(model_dir, exist_ok=True)

# save the PyTorch model state dictionary
model_save_path = os.path.join(model_dir, 'pytorch_model.bin')
torch.save(model.state_dict(), model_save_path)

# Check if the file is saved correctly
!ls -l ./model


total 427764
-rw-r--r-- 1 root root 438024366 Sep 13 05:13 pytorch_model.bin


In [ ]:
import shutil
import os

# Ensure all necessary files are saved in the ./model directory
tokenizer.save_pretrained('./model')
model.config.save_pretrained('./model')

# Copy files from the local Colab environment to Google Drive, overwriting if necessary
model_dir = './model'
drive_path = '/content/drive/My Drive/My Models/bert_model_infosys+TCS'
shutil.copytree(model_dir, drive_path, dirs_exist_ok=True)

# Download the files to local machine
from google.colab import files
files.download('./model/config.json')
files.download('./model/pytorch_model.bin')
files.download('./model/tokenizer_config.json')
files.download('./model/vocab.txt')


#Chatbot

In [71]:
# Load the saved model and tokenizer
model_path = '/content/drive/My Drive/My Models/bert_model_infosys+TCS'
tokenizer_path = '/content/drive/My Drive/My Models/bert_model_infosys+TCS'

model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [72]:
import torch

# Prediction function with extended years
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=128)
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

    # Apply sigmoid function to get probabilities and then round them to get predicted classes
    probabilities = torch.sigmoid(logits)
    predicted_classes = torch.round(probabilities).squeeze().tolist()

    # Matching the question to the filtered DataFrame
    matched_row = filtered_df[filtered_df['Description'].str.lower().apply(lambda x: any(keyword in x for keyword in text.lower().split()))].iloc[0] if not filtered_df[filtered_df['Description'].str.lower().apply(lambda x: any(keyword in x for keyword in text.lower().split()))].empty else None

    if matched_row is not None:
        amounts = {year: matched_row[f'Amount_{year}'] for year in range(2019, 2024)}
    else:
        amounts = {year: "N/A" for year in range(2019, 2024)}

    return predicted_classes, amounts


In [79]:
def predict(text):
    # Normalize the input text to lowercase for better matching
    normalized_text = text.lower()

    # Determine the company and year from the user input
    company = 'infosys' if 'infosys' in normalized_text else 'tcs'
    year = next((y for y in range(2019, 2024) if str(y) in normalized_text), None)

    if not year:
        return None, "Please specify a valid year in your query."

    # Attempt to find a matching row based on the company and year
    matched_row = filtered_df[filtered_df['Description'].str.contains(company, case=False, na=False)]
    if matched_row.empty:
        return None, f"No data found for {company}."

    amount = matched_row[f'Amount_{year}'].iloc[0] if not matched_row.empty else "N/A"

    return amount, f"The {year} amount for {company} is {amount}."

# Adjust your chatbot function to use this new predict method
def chatbot():
    print("Bot: Hello! I am your financial assistant. Ask me about the financial data. (Type 'exit' to quit)")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Bot: Goodbye!")
            break

        amount, response = predict(user_input)
        print(f"Bot: {response}")

# Start the chatbot
chatbot()


Bot: Hello! I am your financial assistant. Ask me about the financial data. (Type 'exit' to quit)
You: What was the total asset value for TCS in 2022?
Bot: The 2022 amount for tcs is 6,861.
You: How have the assets of TCS changed from 2019 to 2023?
Bot: The 2019 amount for tcs is 0.
You: What was the investment income for TCS in 2022?
Bot: The 2022 amount for tcs is 6,861.
You: exit
Bot: Goodbye!
